In [1]:
import numpy as np
from mindspore import context
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceFieldBase
from mindsponge import DynamicUpdater

from mindsponge.potential import BondEnergy, AngleEnergy
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.function import VelocityGenerator
from mindsponge.control import LeapFrog, BerendsenThermostat

## 图模式选择与GPU选择

一般情况下，使用MindSpore的静态图模式GRAPH_MODE会取得更好的速度增益，而动态图模式PYNATIVE_MODE更多的被用于进行Debug。在硬件平台上，如果没有华为的Ascend专用硬件，一般都是在GPU上运行，才有一定的速度保障。如果有多张卡，可以通过device_id进行选择。

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

## 创建分子对象

一个分子，可以是三个原子组成的水分子，也可以是上万个原子组成的蛋白质。在MindSponge中，为不同类型的分子提供了多种定义形式。当定义一个小体系$H_2O$时，我们可以直接用常规的列表来定义分子中所包含的原子种类、原子坐标以及键连信息等。在后续的案例中我们也会讲到，还可以通过配置文件等方式来进行定义，这两种方式都是支持的。

In [3]:
system = Molecule(atoms=['O', 'H', 'H'],
                  coordinate=[[0, 0, 0], [0.1, 0, 0], [-0.0333, 0.0943, 0]],
                  bond=[[[0, 1], [0, 2]]])

## 能量项定义

不同的分子体系会有不同的能量项的需求。比如当前定义的$H_2O$分子，因为只有3个原子，因此涉及到的能量项只有近程相互作用中的键相互作用和角相互作用：

$$
E_{bond}=\frac{1}{2}k_b(l-l_0)^2\\
E_{angle}=\frac{1}{2}k_a(a-a_0)^2
$$

In [4]:
bond_energy = BondEnergy(index=system.bond,
                         force_constant=[[345000, 345000]],
                         bond_length=[[0.1, 0.1]])

In [5]:
angle_energy = AngleEnergy(index=[[1, 0, 2]],
                           force_constant=[[383]],
                           bond_angle=[[109.47 / 180 * np.pi]])

In [6]:
potential = ForceFieldBase(energy=[bond_energy, angle_energy])

## 速度生成器

根据指定的温度，我们可以计算出系统平均运动动能，进而通过标准正态分布生成随机的一组原子运动速度，用于初始化系统。

In [7]:
vgen = VelocityGenerator(300)
velocity = vgen(system.coordinate.shape, system.atom_mass)

## 动力学积分器

如果不对系统施加任何的限制，任由系统去演化的话，就只需要配置integrator积分器，常见的算法有LeapFrog与VelocityVerlet，在MindSponge的control中都是支持的。通过前面配置的各项势能项，利用MindSpore框架的自动微分，就可以计算出当前受力以及下一步的位移，进而实现分子系统的演化。

如果需要对系统施加以一定的控制，比如控制系统的温度，或者是控制系统的压强，在MindSponge的control中可以调用BerendsenThermostat、Barostat等算法，来跑一个NVT的系统，避免系统演化中可能出现的温度震荡现象。

后续在MindSponge的新版本中，还会对LINCS、SETTLE等键长约束算法进行支持，可以在动力学演化的过程中确保不会出现不合理的键长拉伸，最大程度上的保障分子系统键连关系的合理性。

In [8]:
opt = DynamicUpdater(system,
                     integrator=LeapFrog(system),
                     thermostat=BerendsenThermostat(system, 300),
                     time_step=1e-3,
                     velocity=velocity)

In [9]:
md = Sponge(system, potential, opt)

## 信息回调

基于MindSpore的信息回调系统CallBack，我们可以创建一些符合自己需求的信息回调机制，比如使用RunInfo在屏幕上对系统能量进行输出，或者是通过WriteH5MD将轨迹输出到hdf5/h5md文件中。

关于h5md格式的轨迹输出，可以使用改进版的VMD-h5mdplugin进行可视化，相关安装和说明链接为：https://gitee.com/helloyesterday/VMD-h5mdplugin

In [10]:
run_info = RunInfo(10)
cb_h5md = WriteH5MD(system, 'tutorial_b01.h5md', save_freq=10, write_velocity=True, write_force=True)

In [11]:
md.run(1000, callbacks=[run_info, cb_h5md])

Step: 0, E_pot: 3.2556585e-05, E_kin: 2.2424183, E_tot: 2.242451, Temperature: 89.900314
Step: 10, E_pot: 0.08826751, E_kin: 1.9507257, E_tot: 2.0389931, Temperature: 78.20613
Step: 20, E_pot: 0.33794853, E_kin: 1.5694458, E_tot: 1.9073944, Temperature: 62.920315
Step: 30, E_pot: 0.7073656, E_kin: 1.1653502, E_tot: 1.8727157, Temperature: 46.719803
Step: 40, E_pot: 1.1276457, E_kin: 0.8170904, E_tot: 1.9447361, Temperature: 32.757793
Step: 50, E_pot: 1.5250387, E_kin: 0.58905756, E_tot: 2.1140962, Temperature: 23.615782
Step: 60, E_pot: 1.8253816, E_kin: 0.5274784, E_tot: 2.35286, Temperature: 21.147024
Step: 70, E_pot: 1.9750346, E_kin: 0.6455488, E_tot: 2.6205835, Temperature: 25.88056
Step: 80, E_pot: 1.9454645, E_kin: 0.92594516, E_tot: 2.8714097, Temperature: 37.12187
Step: 90, E_pot: 1.7424487, E_kin: 1.3190627, E_tot: 3.0615115, Temperature: 52.882263
Step: 100, E_pot: 1.4011104, E_kin: 1.7579412, E_tot: 3.1590517, Temperature: 70.47725
Step: 110, E_pot: 0.9859305, E_kin: 2.1624

上述运行结果的轨迹输出如下图所示，该结果展示使用了VMD-h5mdplugin插件，在VMD中对分子运行轨迹进行了可视化。

![](../../docs/tutorial_b01.gif)